In [5]:
#!pip install datasets tqdm pandas

In [6]:
#!pip install sentencepiece

In [7]:
#!pip install wandb

In [8]:
#!pip install transformers

In [9]:
#!pip3 install ipywidgets --user

In [10]:
#!pip3 install requests

In [68]:
# Check to confirm that GPU is available



'GeForce GTX 1080 Ti'

In [1]:
!nvidia-smi

Thu Jan 19 23:08:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 19%   24C    P0    58W / 250W |      0MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 17%   

In [3]:
import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.set_device(0)
n_gpu = torch.cuda.device_count()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(n_gpu)
torch.cuda.get_device_name(0)

1


'GeForce GTX 1080 Ti'

In [4]:
torch.cuda.device_count()

1

In [5]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import warnings 
warnings.filterwarnings('ignore')

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
# Check we have a GPU and check the memory size of the GPU
!nvidia-smi

Thu Jan 19 23:09:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   28C    P2    52W / 250W |      2MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 17%   

In [7]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /DATA/gupta92/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
#Set a seed
import random
import numpy as np
import torch
import datasets

In [9]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [10]:
#Loading dataset

In [11]:
pd.set_option('display.max_colwidth', None)

In [35]:
df = pd.read_csv('DATA/english/c4_200m_550k.csv')
df = df.head(50000)
df.shape

(50000, 2)

In [36]:
df.head()

,input,output
0,The steps below describe how to remove data for one or more specifies areas and how to put on the data from a snapshot to the index,The steps below describe how to remove data for one ore more specific areas and how to put back the data from a snapshot to the index.
1,When I wake up it\'s usually comes out dreamsI\'m thinking so my thoughts are very weird.,When I wake up it\'s usually dreams I\'m thinking about so my thoughts are very weird.
2,One of the cardinal factors to be considered trying to decide on which kind of shipping to customer settle is the! market difference.,One of the cardinal factors to consider when trying to decide on which kind of shipping to settle for is the market difference.
3,Answers » Regions » Is in Nagorno-Karabakt region that part in Armenia?,Answers » Regions » Is Nagorno-Karabakh region part of Armenia?
4,Flaneuring in fun at maple creek SK!,Flaneuring Fun in Maple Creek SK!


In [37]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [38]:
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

loading file spiece.model from cache at /DATA/gupta92/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers":

In [39]:
def calc_token_len(example):
    #return len(tokenizer(example).input_ids)
    return len(tokenizer(example[:512]).input_ids)

In [40]:
#Train - Test Split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((45000, 2), (5000, 2))

In [41]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [42]:
test_df.head()

,input,output,input_token_len
33553,"1 office space 175,111 square feet","1 office space 175,111 square feet.",9
9427,I request the input Danielle!,I appreciate the input Danielle!,8
199,"In the last day, we will transfer you in to either Airport or turn you back to Tangier. End of our service.","On the last day, we will transfer you either to the Airport, or turn you back to Tangier. End of our service.",27
12447,It does not use a 1 32 bit Download free screensavers cineravors time clock mode is The # 1 Free Employee Time Clock with advanced online scheduling and payroll timereproduction clockdisplayed sea Payroll cp free ever download for using forever Link in info area 1980x1080 MOV Alarm Clocks Online Alarm Clock Time Zone Clocks Military Time Military Time Converter Analog Alarm Clock White Noise Generator Video Alarm ountClock CaClock Countdown Timers Countdown.,It does not use a. 1 32 bit]. Download Free Screensavers 3D. Time Clock Hub is The # 1 Free Employee Time Clock with Advanced Online Scheduling and Payroll Timelapse clock: Clock Time Lapse Video Download CC Free to Use Forever Link in Info Area 9mb 1980x1080 MOV Alarm Clocks Online Alarm Clock Time Zone Clocks Military Time Military Time Converter Analog Alarm Clock White Noise Generator Video Alarm Clock Countdown Timers Countdown.,111
39489,The rise of Chinese supertech giant: What is the implication of food companies?,The Rise of China's Supertech Giants: What Is the Implication for Food Companies?,18


In [43]:
test_df['input_token_len'].describe()

count    5000.000000
mean       32.904400
std        22.308106
min         5.000000
25%        17.000000
50%        27.000000
75%        42.000000
max       307.000000
Name: input_token_len, dtype: float64

In [44]:
#We will use a token length of 64 since it will cover the vast majority of examples
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [45]:
test_dataset

Dataset({
    features: ['input', 'output', 'input_token_len', '__index_level_0__'],
    num_rows: 5000
})

In [46]:
#Load the Dataset
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [47]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[56])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 24
attention_mask 24
labels 22
{'input_ids': [2449, 691, 91, 128, 126, 1726, 4113, 44, 37, 10144, 8257, 3111, 44, 8, 5331, 3254, 11347, 1166, 44, 8561, 26, 9, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [2449, 691, 91, 128, 126, 1726, 3598, 44, 37, 10144, 8257, 3111, 44, 8, 5331, 3254, 11347, 1166, 30, 1701, 5, 1]}


In [48]:
#Define Evaluator
#!pip install rouge_score

In [49]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [50]:
#Train Model
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [51]:
# defining training related arguments
batch_size = 8
args = Seq2SeqTrainingArguments(output_dir="models/english/c4_200m/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 100,
                        save_steps = 100,
                        load_best_model_at_end=True,
                        logging_dir="logs",
                        report_to = None    )
                        #report_to="wandb")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [52]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /DATA/gupta92/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
#!pip install wandb

In [54]:
# defining trainer using 🤗

trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

Using cuda_amp half precision backend


In [55]:
#wandb login
#bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9

In [56]:
#bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
"""

import wandb
wandb.init(project="gec", entity="gupta92")
"""
print()

In [57]:
#Wandb API Key: bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

***** Running training *****
  Num examples = 45000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 6
  Total optimization steps = 937


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.788000,0.655472,71.211300,60.586800,70.458600,70.489300,17.329200


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to models/english/c4_200m/weights/checkpoint-500
Configuration saved in models/english/c4_200m/weights/checkpoint-500/config.json
Model weights saved in models/english/c4_200m/weights/checkpoint-500/pytorch_model.bin
tokenizer config file saved in models/english/c4_200m/weights/checkpoint-500/tokenizer_config.json
Special tokens file saved in models/english/c4_200m/weights/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from models/english/c4_200m/weights/checkpoint-500 (score: 0.6554718017578125).


TrainOutput(global_step=937, training_loss=0.7505102330617996, metrics={'train_runtime': 1385.8191, 'train_samples_per_second': 32.472, 'train_steps_per_second': 0.676, 'total_flos': 3050079774474240.0, 'train_loss': 0.7505102330617996, 'epoch': 1.0})

In [58]:
#Save the trained Model
trainer.save_model('t5_gec_model_english_new')

Saving model checkpoint to t5_gec_model_english_new
Configuration saved in t5_gec_model_english_new/config.json
Model weights saved in t5_gec_model_english_new/pytorch_model.bin
tokenizer config file saved in t5_gec_model_english_new/tokenizer_config.json
Special tokens file saved in t5_gec_model_english_new/special_tokens_map.json


In [59]:
!zip -r 't5_gec_model_english_new.zip' 't5_gec_model_english_new'

  adding: t5_gec_model_english_new/ (stored 0%)
  adding: t5_gec_model_english_new/spiece.model (deflated 48%)
  adding: t5_gec_model_english_new/pytorch_model.bin (deflated 9%)
  adding: t5_gec_model_english_new/special_tokens_map.json (deflated 86%)
  adding: t5_gec_model_english_new/config.json (deflated 62%)
  adding: t5_gec_model_english_new/training_args.bin (deflated 48%)
  adding: t5_gec_model_english_new/tokenizer_config.json (deflated 82%)


In [60]:
#Testing

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 't5_gec_model_english_new'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model1 = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)
model=model.to(torch_device)

loading file spiece.model from cache at /DATA/gupta92/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers":

In [61]:
def correct_grammar(input_text,num_return_sequences):
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    translated = model1.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [62]:
text = 'She are moving here.'
print(correct_grammar(text, num_return_sequences=2))

['She is moving here.', 'She are moving here.']


In [63]:
text = 'Cat drink milk.'
print(correct_grammar(text, num_return_sequences=4))

['Cat drink milk.', 'Cat drinks milk.', 'Cats drink milk.', 'Cat drinking milk.']


In [65]:
text = 'You going home.'
print(correct_grammar(text, num_return_sequences=4))

['You going home.', "You're going home.", 'You go home.', 'You are going home.']


In [66]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'deep-learning-analytics/GrammarCorrector'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)


loading file spiece.model from cache at /DATA/gupta92/.cache/huggingface/hub/models--deep-learning-analytics--GrammarCorrector/snapshots/6ca90bd771c373a0542d4257a5c34d26cd0d3c59/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--deep-learning-analytics--GrammarCorrector/snapshots/6ca90bd771c373a0542d4257a5c34d26cd0d3c59/special_tokens_map.json
loading file tokenizer_config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--deep-learning-analytics--GrammarCorrector/snapshots/6ca90bd771c373a0542d4257a5c34d26cd0d3c59/tokenizer_config.json
loading configuration file config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--deep-learning-analytics--GrammarCorrector/snapshots/6ca90bd771c373a0542d4257a5c34d26cd0d3c59/config.json
Model config T5Config {
  "_name_or_path": "deep-learning-analytics/GrammarCorrector",
  "architectures": [
    "T5ForConditionalGener

In [4]:
def correct_grammar(input_text,num_return_sequences):
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [68]:
text = 'He are moving here.'
print(correct_grammar(text, num_return_sequences=2))

['He is moving here.', "He's moving here."]


In [69]:
text = 'Cat drinked milk.'
print(correct_grammar(text, num_return_sequences=3))

['Cat drinks milk.', 'Cat drink milk.', 'Cat drank milk.']


In [1]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'deep-learning-analytics/GrammarCorrector'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
#output_dir = '/content/drive/My Drive/saved_model/BERT_model'
output_dir = 't5_gec_model_english_new'


print(output_dir)

tokenizer = T5Tokenizer.from_pretrained(output_dir)
model = T5ForConditionalGeneration.from_pretrained(output_dir).to(torch_device)

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


t5_gec_model_english_new


In [3]:
#model

In [2]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'deep-learning-analytics/GrammarCorrector'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
#output_dir = '/content/drive/My Drive/saved_model/BERT_model'
output_dir = 't5_gec_model_english_new'


print(output_dir)

tokenizer = T5Tokenizer.from_pretrained(output_dir)
model = T5ForConditionalGeneration.from_pretrained(output_dir).to(torch_device)


t5_gec_model_english_new


In [5]:
text = 'Cat drinked milk.'
print("Incorrect Sentence:", text)
predicted_s = correct_grammar(text, num_return_sequences=4)[1]
print("Predicted Sentence:",predicted_s)
print("___________________________________________________________________________")

text = 'I doing my M.Tech Project.'
print("Incorrect Sentence:", text)
predicted_s = correct_grammar(text, num_return_sequences=4)[1]
print("Predicted Sentence:",predicted_s)
print("___________________________________________________________________________")

text = 'You is a kind person.'
print("Incorrect Sentence:", text)
predicted_s = correct_grammar(text, num_return_sequences=4)[1]
print("Predicted Sentence:",predicted_s)
print("___________________________________________________________________________")

Incorrect Sentence: Cat drinked milk.
Predicted Sentence: Cat drank milk.
___________________________________________________________________________
Incorrect Sentence: I doing my M.Tech Project.
Predicted Sentence: I'm doing my M.Tech Project.
___________________________________________________________________________
Incorrect Sentence: You is a kind person.
Predicted Sentence: You're a kind person.
___________________________________________________________________________


In [74]:
text = 'Cat drinked milk.'
print(correct_grammar(text, num_return_sequences=4)[1])

Cat drank milk.


In [ ]:
text = 'Cat drinked milk.'
print(correct_grammar(text, num_return_sequences=4)[1])


In [75]:
text = 'I doing my M.Tech Project.'
print(correct_grammar(text, num_return_sequences=4)[1])

I'm doing my M.Tech Project.


In [76]:
text = 'You is a kind person.'
print(correct_grammar(text, num_return_sequences=4)[1])

You're a kind person.


/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                    train/epoch ▁
wandb:              train/global_step ▁
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                    train/epoch 0.96
wandb:              train/global_step 18
wandb:               train/total_flos 58888131010560.0
wandb:               train/train_loss 1.25195
wandb:            train/train_runtime 22.9697
wandb: train/train_samples_per_second 39.182
wandb:   train/train_steps_per_second 0.784
wandb: 
wandb: You can sync this run to the cloud by running:
wandb: wandb sync /DATA/gupta92/wandb/offli